# do a transitive query of a wikidata concept and search CU Experts elastic endpoint for all people who have this concept in their elastic document

### Use anaconda shell to add dependencies. eg: conda install -c conda-forge sparqlwrapper

In [164]:
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import namedtuple
import json
import requests

endpoint_url = "https://query.wikidata.org/sparql"
ES_person_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search"
ES_pub_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search"
search_term = "artificial intelligence"
person_termlist = dict()
query = """SELECT ?subitem ?subitemLabel
WHERE {

  ?item ?label \"""" + search_term + """\"@en.
    ?item wdt:P279* wd:Q151885.
  ?subitem wdt:P279+ ?item.
  SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\". }
  } """



In [165]:
def WdataSearch(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [166]:
def ESsearch(uri, term):
    """Simple Elasticsearch Query"""
    query = json.dumps({
        "query": {
#            "match": {
#                "researchArea.name.exact": term
#            }
             "query_string": { "query": term }
        }
    })
    response = requests.get(uri, data=query)
    results = json.loads(response.text)
    return results



In [167]:
print ("Only displaying results with a search score > .5")
print (search_term)
ESresults = ESsearch(ES_person_endpoint_url,search_term)
for eresult in ESresults['hits']['hits']:
      if eresult["_score"] > .5:
        #person=personterms(search_term,eresult["_score"])
        #person.name = eresult["_source"]["name"]
        termlist=[]
        termscore={search_term,eresult["_score"]}
        termlist.append(termscore)
        person_termlist[eresult["_source"]["name"]]=termlist
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])

Wresults = WdataSearch(endpoint_url, query)


for result in Wresults["results"]["bindings"]:
   print ("\n")
   print (result['subitemLabel']['value'], result['subitem']['value'])
   ESresults = ESsearch(ES_person_endpoint_url,result['subitemLabel']['value'])
   print ("Total people who match in CU Experts: ", ESresults['hits']['total'])
   for eresult in ESresults['hits']['hits']:
      if eresult["_score"] > .5:
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])
        termscore={result['subitemLabel']['value'],eresult["_score"]}
        if eresult["_source"]["name"] in person_termlist:
            person_termlist[eresult["_source"]["name"]].append(termscore)
        else:
            termlist=[]
            termlist.append(termscore)
            person_termlist[eresult["_source"]["name"]]=termlist

print ("\nList of persons and thier terms")
for result in person_termlist:
    print(result,": ")
    for terms in person_termlist[result]:
       print("\t", terms)


Only displaying results with a search score > .5
artificial intelligence
0.9771342 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
0.8549925 Ward, Wayne Hinson https://experts.colorado.edu/individual/fisid_114680
0.85185 Yeh, Tom https://experts.colorado.edu/individual/fisid_151584
0.7390373 Roncone, Alessandro https://experts.colorado.edu/individual/fisid_164509
0.65666205 Burke, Robin D. https://experts.colorado.edu/individual/fisid_165005
0.6334605 Martin, James H https://experts.colorado.edu/individual/fisid_100495
0.6334605 Mathys, Peter https://experts.colorado.edu/individual/fisid_100084
0.61070895 Repenning, Alexander https://experts.colorado.edu/individual/fisid_104946
0.60846424 Foltz, Peter https://experts.colorado.edu/individual/fisid_141960
0.60846424 Trivedi, Ashutosh https://experts.colorado.edu/individual/fisid_156589


machine learning http://www.wikidata.org/entity/Q2539
Total people who match in CU Experts:  66
0.63453895 Tsai, Janet Yi-Jen https:/

Total people who match in CU Experts:  3


tokenization http://www.wikidata.org/entity/Q2438971
Total people who match in CU Experts:  0


text simplification http://www.wikidata.org/entity/Q3484781
Total people who match in CU Experts:  3


computer-based question classification http://www.wikidata.org/entity/Q46346005
Total people who match in CU Experts:  356


biomedical natural language processing http://www.wikidata.org/entity/Q51751772
Total people who match in CU Experts:  395


cognitive neuroscience http://www.wikidata.org/entity/Q1138951
Total people who match in CU Experts:  125
1.1692705 Wang, Kai https://experts.colorado.edu/individual/fisid_156598
0.97744894 Miyake, Akira https://experts.colorado.edu/individual/fisid_107321
0.9080568 Banich, Marie https://experts.colorado.edu/individual/fisid_120646
0.87468535 Watson, Karli K https://experts.colorado.edu/individual/fisid_155058
0.8707323 D'Mello, Sidney https://experts.colorado.edu/individual/fisid_159117
0.85057694 Krag

Total people who match in CU Experts:  197
0.65406764 Wang, Kai https://experts.colorado.edu/individual/fisid_156598
0.59777766 Vigers, Alison Jane https://experts.colorado.edu/individual/fisid_142378
0.54916245 Munakata, Yuko https://experts.colorado.edu/individual/fisid_125036
0.526251 Miyake, Akira https://experts.colorado.edu/individual/fisid_107321
0.5176775 Banich, Marie https://experts.colorado.edu/individual/fisid_120646


independent component analysis http://www.wikidata.org/entity/Q1259145
Total people who match in CU Experts:  137


archetypal analysis http://www.wikidata.org/entity/Q25051436
Total people who match in CU Experts:  135


boosting http://www.wikidata.org/entity/Q466303
Total people who match in CU Experts:  0


Boltzmann machine http://www.wikidata.org/entity/Q194706
Total people who match in CU Experts:  8


winner-take-all http://www.wikidata.org/entity/Q769931
Total people who match in CU Experts:  3


Long Short-Term Memory http://www.wikidata.org/entity/